In [3]:
'''
Skeleton code for resolution theorem prover for propositional logic

Yoonsuck Choe
10/20/2021

clause representation:

  [<positive-proposition-list>, <negative-proposition-list>]

proposition-list representation:

  [ 'p', 'q', 'r' ]

  Note: 'p', 'q', 'r', 's' are propositions : CASE SENSITIVE!

Example clause:

  [['p', 'q'] ['r']] : This is clause P \/ Q \/ ~R

'''

DEBUG=True

#---------------------------------------------------------------------
def rm_item(lst, item):
#---------------------------------------------------------------------
  '''
  function: rm_pred()

  remove item from list
  - return [] if empty, not None
  '''
  lst.remove(item)

  if lst == None:
    return []
  else:
    return lst


#---------------------------------------------------------------------
def mk_unique(clause):
#---------------------------------------------------------------------
  '''
  remove redundant propositions in the clause
  '''
  pos = clause[0]
  neg = clause[1]

  pos = list(set(pos))
  neg = list(set(neg))

  return [pos, neg]

#---------------------------------------------------------------------
def resolve(clause1, clause2):
#---------------------------------------------------------------------
  '''
  function: resolve()

  [['p', 'q'] ['r']] : clause P \/ Q \/ ~R

  resolve clauses c1 and c2
  - if resolvable returns resolvent
  - if not resolvable returns False
  - returns empty clause [[], []] when Falsified

  CAUTION: this is not perfect.
  '''

  # remove any redundant propositions in the clauses
  c1 = mk_unique(clause1)
  c2 = mk_unique(clause2)

  count = 0

  # print clauses
  if DEBUG:
    print("resolving: "+str(c1)+" and "+str(c2))

  # c1 pos vs. c2 neg
  for p1 in c1[0]:
    for p2 in c2[1]:
      if p1==p2:
        c1 = [rm_item(c1[0],p1), c1[1]]
        c2 = [c2[0], rm_item(c2[1],p2)]
        count = count+1

  # c2 pos vs. c1 neg
  for p1 in c1[1]:
    for p2 in c2[0]:
      if p1==p2:
        c1 = [c1[0], rm_item(c1[1],p1)]
        c2 = [rm_item(c2[0],p2), c2[1]]
        count = count+1

  # check for multiple matches or no match and abort if so.
  if count>1 or count==0:
    return False

  # compute resolvent
  c1[0].extend(c2[0])
  c1[1].extend(c2[1])

  # make unique and return

  return mk_unique(c1)



#---------------------------------------------------------------------
def null_p(clause):
#---------------------------------------------------------------------
  '''
  function null_p()

  check if "False" is derived
  - returns True if empty clause
  - returns False if non-empty clause
  '''

  if (type(clause) is list) and len(clause[0]) + len(clause[1]) == 0:
    return True
  else:
    return False



#---------------------------------------------------------------------
def print_clause(clause):
#---------------------------------------------------------------------
  '''
  function print_clause

  See function resolve() for clause representation.

  '''

  s = ""
  for pos in clause[0]:
    s = s+pos+"\/"

  for neg in clause[1]:
    s = s+"~"+neg+"\/"

  if len(s)==0:
    print("NULL")
  else:
    n=len(s)
    print(s[0:(n-2)])

#------------------------------------------------------
# test resolve() function
#------------------------------------------------------

print("\ntest1")
test1 = resolve([['p', 'q'], ['r']], [[], ['p']])
print("resolvent = "+str(test1))
print("null? "+str(null_p(test1)))

print("\ntest2")
test2 = resolve([['p'], []], [[], ['p']])
print("resolvent = "+str(test2))
print("null? "+str(null_p(test2)))

print("\ntest3")
test3 = resolve([['p', 'q'], ['r']], [['r'], ['p']])
print("resolvent = "+str(test3))
print("null? "+str(null_p(test3)))

print("\ntest4")
test4 = resolve([['p', 'q'], ['r']], [['r'], []])
print("resolvent = "+str(test4))
print("null? "+str(null_p(test4)))


print("\ntest5")
test5 = resolve([['p', 'q'], ['r']], [['p', 'r'], []])
print("resolvent = "+str(test5))
print("null? "+str(null_p(test5)))


# edge case not being handled correctly: p \/ q \/ ~r \/ ~r  vs. p \/ r should give p \/ q, not p \/ q \/ ~r
# - now this is fixed, with the use of mk_unique()
print("\ntest6")
test5 = resolve([['p', 'q'], ['r', 'r']], [['p', 'r'], []])
print("resolvent = "+str(test5))
print("null? "+str(null_p(test5)))



test1
resolving: [['p', 'q'], ['r']] and [[], ['p']]
resolvent = [['q'], ['r']]
null? False

test2
resolving: [['p'], []] and [[], ['p']]
resolvent = [[], []]
null? True

test3
resolving: [['p', 'q'], ['r']] and [['r'], ['p']]
resolvent = False
null? False

test4
resolving: [['p', 'q'], ['r']] and [['r'], []]
resolvent = [['p', 'q'], []]
null? False

test5
resolving: [['p', 'q'], ['r']] and [['r', 'p'], []]
resolvent = [['p', 'q'], []]
null? False

test6
resolving: [['p', 'q'], ['r']] and [['r', 'p'], []]
resolvent = [['p', 'q'], []]
null? False


In [5]:
'''
Using the resolve() function and the null_p function, implement the two-pointer method for propositional logic theorem proving
- use set of support

theorem representation:

[ [<index> <clause1>], [<index> <clause2>], ... ]

index : integer

clause representation:

[<positive-proposition-list>, <negative-proposition-list>]

proposition-list representation:

[ 'p', 'q', 'r' ]

'''

# define your theorem, as a set of indexed clauses

theorem = [    # pos-list      # neg-list
           [0, [['p','q'],    ['r']]],       # clause 0     P \/ Q \/ ~R
           [1, [['s'],        ['q']]],       # clause 1     S \/ ~Q
           [2, [['t'],        ['s']]],       # clause 2     T \/ ~S
           [3, [[],           ['p']]],       # clause 3     ~P   - negated conclusion starts here (goal clause index = 3)
           [4, [['r'],        []]],          # clause 4     R
           [5, [[],           ['t']]]        # clause 5     ~T
          ]

theorem2 = [ # pos-list      # neg-list
           [0, [['i'],        ['m']]],       # clause 0      I \/ ~M
           [1, [['m'],        ['i']]],       # clause 1      M \/ ~I
           [2, [['m' , 'l'],  []]],          # clause 2      M \/ L
           [3, [['h'],        ['i']]],       # clause 3      H \/ ~I
           [4, [['h'],        ['l']]],       # clause 4      H \/ ~L
           [5, [['g'],        ['h']]],       # clause 5      G \/ ~H
           [6, [[],           ['m']]]        # clause 6      ~M - negated conclusion starts here (goal clause index = 6)
           ]  # enter the example theorm in the homework

theorem3 = [ # pos-list      # neg-list
           [0, [['i'],        ['m']]],       # clause 0      I \/ ~M
           [1, [['m'],        ['i']]],       # clause 1      M \/ ~I
           [2, [['m' , 'l'],  []]],          # clause 2      M \/ L
           [3, [['h'],        ['i']]],       # clause 3      H \/ ~I
           [4, [['h'],        ['l']]],       # clause 4      H \/ ~L
           [5, [['g'],        ['h']]],       # clause 5      G \/ ~H
           [6, [[],           ['h']]]        # clause 6      ~H - negated conclusion starts here (goal clause index = 6)
           ]  # enter your own example for testing

theorem4 = [    # pos-list      # neg-list
           [0, [['i'],        ['m']]],       # clause 0      I \/ ~M
           [1, [['m'],        ['i']]],       # clause 1      M \/ ~I
           [2, [['m' , 'l'],  []]],          # clause 2      M \/ L
           [3, [['h'],        ['i']]],       # clause 3      H \/ ~I
           [4, [['h'],        ['l']]],       # clause 4      H \/ ~L
           [5, [['g'],        ['h']]],       # clause 5      G \/ ~H
           [6, [[],           ['g']]]        # clause 6      ~G - negated conclusion starts here (goal clause index = 6)
          ]

# item_in_thrm() to check if the clauses already exists in the theorem.
def item_in_thrm(item, thrm):
    return any(resolvent == item for _, resolvent in thrm)

#------------------------------------------------------
# prover() function : you must implement this function
#---------------------------------------------------------------------
def prover(thm, goal):
#---------------------------------------------------------------------
  '''
  function prover: implement this

  two-pointer method, with set of support
  - arguments:
    thm : theorem
    goal : integer index (clause number where the negated conclusion starts)
  - show resolution steps
  - if null_p checks, return True (theorem proven)
  - otherwise return False
  '''

  print('\nprover():\n\nTheorem:')

  for clause in theorem:
    print("clause "+str(clause[0])+": ",end="")
    print_clause(clause[1])

  print('Goal clause index=',end='')
  print(goal)

  j = goal
  step = 0
  while j < len(thm):
    i = 0
    while i < j:
      resolvent = resolve(thm[i][1], thm[j][1])
      print("resolvent = "+str(resolvent))

      if null_p(resolvent):
        print("Success")
        print(step)
        return True

      if resolvent != False and not item_in_thrm(resolvent, thm):
        step = step + 1
        thm.append([len(thm), resolvent])

      i = i+1
    j = j+1

  print(step)
  print("Failure")
  return False


  print('Test resolve: clause 0 and clause 1')
  print_clause(resolve(theorem[0][1], theorem[1][1]))

  print('\n\n--- must print the resolution steps\n\n\n')

  return True

#------------------------------------------------------
# test prover() function
#------------------------------------------------------

# Test prover on example theorem

prover(theorem, 3)

print("======== Resolving for M =========")
prover(theorem2, 6)

print("======== Resolving for H =========")
prover(theorem3, 6)

print("======== Resolving for G =========")
prover(theorem4, 6)

# Test clause printer

print("Testing print_clause()")
print_clause(theorem[0][1])
print_clause([['p'], ['q']])
print_clause([['p','s'], ['q','r']])


prover():

Theorem:
clause 0: p\/q\/~r
clause 1: s\/~q
clause 2: t\/~s
clause 3: ~p
clause 4: r
clause 5: ~t
Goal clause index=3
resolving: [['p', 'q'], ['r']] and [[], ['p']]
resolvent = [['q'], ['r']]
resolving: [['s'], ['q']] and [[], ['p']]
resolvent = False
resolving: [['t'], ['s']] and [[], ['p']]
resolvent = False
resolving: [['p', 'q'], ['r']] and [['r'], []]
resolvent = [['p', 'q'], []]
resolving: [['s'], ['q']] and [['r'], []]
resolvent = False
resolving: [['t'], ['s']] and [['r'], []]
resolvent = False
resolving: [[], ['p']] and [['r'], []]
resolvent = False
resolving: [['p', 'q'], ['r']] and [[], ['t']]
resolvent = False
resolving: [['s'], ['q']] and [[], ['t']]
resolvent = False
resolving: [['t'], ['s']] and [[], ['t']]
resolvent = [[], ['s']]
resolving: [[], ['p']] and [[], ['t']]
resolvent = False
resolving: [['r'], []] and [[], ['t']]
resolvent = False
resolving: [['p', 'q'], ['r']] and [['q'], ['r']]
resolvent = False
resolving: [['s'], ['q']] and [['q'], ['r']]
resol